In [1]:
from deep_translator import GoogleTranslator
import spacy
import services.influencers_sevice as influencers_service
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
import json 
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#visualisation 
import pyLDAvis
import pyLDAvis.gensim

In [163]:
translator = GoogleTranslator(source='auto', target='en')
influencers = influencers_service.get_influencers()
posts=influencers_service.get_posts()
stopwords_en = stopwords.words('english')

In [164]:
def translate_post(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        try:
            translated_content = translator.translate(content)
            print("after tanslation", translated_content)
            influencers_service.update_post(post, key,translated_content)
    
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this caption.")

In [165]:
def translate_hashtags(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        translated_hashtags = []
        for hashtag in hashtags:
            try:
                translated_hashtag = translator.translate(hashtag)
                print(translated_hashtag)
                translated_hashtags.append(translated_hashtag)
                post['hashtags'] = translated_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [166]:
def lemmatize_text(text):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    allowed_postags = ["NOUN", "AD", "VERB", "ADV"]
    doc = nlp(text)
    new_text = []
    for word in doc:
        print(word)
        new_text.append(word.lemma_)
    lemmatized_text = " ".join(new_text)
    return lemmatized_text

In [167]:
def lemmatize_post(posts,key):
    for post in posts: 
        try:
            content = post.get(key)
            lemmatized_content = lemmatize_text(content)
            influencers_service.update_post(post, key , lemmatized_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this hashtag.")

In [168]:
def lemmatize_hashtags(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        lemmatized_hashtags = []
        for hashtag in hashtags:
            try:
                lemmatized_hashtag = lemmatize_text(hashtag)
                lemmatized_hashtags.append(lemmatized_hashtag )
                post['hashtags'] = lemmatized_hashtags
                updated_hashtags.append(hashtag)
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [169]:
def remove_emojis(text):
    if(text):
        print("before emoji removal :",text)
        return unidecode(text)
    return text


In [170]:
def remove_hashtags_emojis(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        for hashtag in hashtags:
            try:
                updated_hashtag = remove_emojis(hashtag)
                print(updated_hashtag)
                updated_hashtags .append(updated_hashtags)
                post['hashtags'] = updated_hashtags
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [171]:
def remove_post_emojis(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        no_emoji_content = remove_emojis(content)
        influencers_service.update_post(post, key, no_emoji_content)

In [172]:
def remove_stopwords(text, stopwords_list):
    words = text.split()
    words = [word for word in words if word not in stopwords_list]
    return ' '.join(words)

In [173]:
def remove_post_stopwords(posts,key):
    for post in posts:
        content = post.get(key)
        try:
            no_stopwords_content = remove_stopwords(content, stopwords_en)
            print(no_stopwords_content)
            influencers_service.update_post(post, key, no_stopwords_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this post")

In [174]:
def remove_hashtags_stop_words(posts):
    updated_hashtags = []
    for post in posts:
        hashtags = post.get('hashtags', [])
        for hashtag in hashtags:
            try:
                print("before:",hashtag)
                updated_hashtag = remove_stopwords(hashtag)
                print("after:",updated_hashtag)
                updated_hashtags .append(updated_hashtag)
                post['hashtags'] = updated_hashtags
            except Exception as e:
                print(f"An exception occurred: {e}. Skipping this hashtag.")
                continue
        influencers_service.update_post(post, 'hashtags', updated_hashtags)

In [175]:
# translate_post(posts,'firstComment')
# translate_post(posts, 'caption')
# translate_hashtags(posts)

In [176]:
remove_post_emojis(posts, 'firstComment')
# remove_post_emojis(posts, 'caption')
# # remove_hashtags_emojis(posts)

None



hey , check dm , I send message recently ..
before emoji removal : hey , check dm , I send message recently ..
reply
before emoji removal : reply
None
Eid Mubarak . happy new year
before emoji removal : Eid Mubarak . happy new year
health drink acceptable , good breakfast , God willing , Hanan
before emoji removal : health drink acceptable , good breakfast , God willing , Hanan
Mashallah
before emoji removal : Mashallah
@fladjimi1 @eya_oueslatii @world_lad @melek_ladjimi @ladjimirayen
before emoji removal : @fladjimi1 @eya_oueslatii @world_lad @melek_ladjimi @ladjimirayen
@melek_ladjimi @ladjimirayen @ladjimi_donia @the_world @eya_oueslatii
before emoji removal : @melek_ladjimi @ladjimirayen @ladjimi_donia @the_world @eya_oueslatii
None
None
None
ya 3tek 3asba lik ouleha sawsen
before emoji removal : ya 3tek 3asba lik ouleha sawsen
clothed naked woman enter Paradise scent dull . appointment God . openly commit sin God , reckoning difficult . keep go , keep go . indeed , Ramadan

In [177]:
# remove_post_stopwords(posts, 'firstComment')
# remove_post_stopwords(posts, 'caption')
# remove_hashtags_stop_words(posts)

In [178]:
# lemmatize_post(posts,'firstComment')
# lemmatize_post(posts, 'caption')
# lemmatize_hashtags(posts)        

In [179]:
# print("doooooooooone")

Merging the captions and first comment of all posts in one text

In [180]:
def merge_content_posts(posts) : 
    #concatenation of caption and firstComment of all selected posts
    merged_content = ""
    for post in posts:
         #extract caption text
         caption = post.get('caption')
         #extract firstComment ext
         # firstComment = post.get('firstComment', [])
         firstComment="me"
         #concatenate caption and firstComment in one string
         if (caption and firstComment):
            merged_content += caption + " " + firstComment
    return merged_content

Extracting words from merged content

In [181]:
def gen_words(text):
    words=[]
    new = gensim.utils.simple_preprocess(text, deacc=True)    
    words.append(new)
    return words


In [182]:
def gen_bow(words):
    id2word = corpora.Dictionary(words)
    corpus=[]
    for word in words: 
        new = id2word.doc2bow(word)
        corpus.append(new)
    return  corpus

In [183]:
def make_bigrams(texts,bigram):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts,bigram,trigram):
    return [trigram[bigram[doc]] for doc in texts]

In [184]:
# # posts=posts[0:3] #for rapid testing we only use three posts
# merge=merge_content_posts(posts)
# # print(merge)
# words=gen_words(merge)
# # print(words)
# 
# ##generate bigrams and tigrams##
# bigram_phrases = gensim.models.Phrases(words, min_count=5, threshold=100)
# trigram_phrases = gensim.models.Phrases(bigram_phrases[words], threshold=100)
# bigram = gensim.models.phrases.Phraser(bigram_phrases)
# trigram = gensim.models.phrases.Phraser(trigram_phrases)
# data_bigrams = make_bigrams(words,bigram)
# data_bigrams_trigrams = make_trigrams(data_bigrams,bigram,trigram)


In [185]:
# from gensim.parsing.preprocessing import STOPWORDS
# texts = data_bigrams_trigrams #for easier typing
# custom_stopwords = {'god','bless','tunisia','la','ly','et','al','fy','mn','mei'}  # Add any additional stop words to this set
# stopwords = STOPWORDS.union(custom_stopwords)
# filtered_texts = []
# for text in texts:
#     filtered_text = [word for word in text if word not in stopwords]
#     filtered_texts.append(filtered_text)
# texts=filtered_texts
# # print(texts)

In [186]:
# # # #TF-IDF REMOVAL
# from gensim.models import TfidfModel
# # 
# id2word = corpora.Dictionary(texts)
# corpus = [id2word.doc2bow(text) for text in texts]
# tfidf = TfidfModel(corpus, id2word=id2word,smartirs='ntc')
# 
# low_value =0.0005
# words  = []
# words_missing_in_tfidf = []
# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = []
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words+words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow
#     
# # print(corpus)

In [187]:
# lda_model=gensim.models.ldamodel.LdaModel(
#     corpus=corpus,
#     id2word=id2word,
#     num_topics=5,
#     random_state=50,
#     chunksize=1000,
#     update_every=1,
#     passes=1,
#     alpha=0.01,
#     eta=0.002,
# )

In [188]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30,lambda_step=0.01)
# vis